In [ ]:
import os
import re
import vtk
import ants
import matplotlib
import numpy as np
import os.path as op
import nibabel as nib
import matplotlib.pyplot as plt

from fury import actor, window
from dipy.data import get_sphere

In [ ]:
paths_data = op.join("/path", "to", "data")
paths_save = op.join("paths", "to", "figure01")
os.makedirs(paths_save, exist_ok = True)

In [ ]:
plt.rcParams.update({
  "text.usetex": False,
  "font.family": "Helvetica",
  "font.size": 14
})

def create_vtk_colormap(cmap, scale_range = (0, 1), n = 256):
    # get colors from matplotlib colormap 
  colors = matplotlib.colormaps[cmap]
  colors = colors(np.linspace(0, 1, n))[..., :3]
    
  # create VTK lookup table
  lut = vtk.vtkLookupTable()
  lut.SetNumberOfTableValues(n)
  lut.SetRange(scale_range[0], scale_range[1])
  
  # assign colors in vtk lookup table
  for i in np.arange(n):
    r, g, b = colors[i]
    lut.SetTableValue(i, r, g, b, 1.0)  # RGBA
  lut.Build()

  return lut

In [ ]:
# fodf reliability 
participant_info = [
  ["sub-XXX", "Original"], # low reliability
  ["sub-XXX", "FWE"], # medium reliability
  ["sub-XXX", "MSMT"] # high reliability
]

x_min = 18; x_max = x_min + 20; nx = x_max - x_min # 20 width
y_min = 17; y_max = y_min + 20; ny = y_max - y_min # 20 width
z_slice = 37; nz = 1 

corr2_cmap = "gray"
corr2_range = (0, 1) # range of corr2 values
corr2_lut = create_vtk_colormap(corr2_cmap, scale_range = corr2_range)

scene = window.Scene() # open scene
for participant, method in participant_info: # for each participant
  # define participant data directory
  data_dir = op.join(paths_data, participant)

  # load brain mask image (reference for diffusion sampling)
  mask_fname = f"{participant}_space-ACPC_desc-brain_mask.nii.gz"
  mask_image = ants.image_read(op.join(data_dir, mask_fname))

  # load anatomical images (to be resampled to diffusion space)
  dseg_fname = f"{participant}_space-ACPC_desc-aseg_dseg.nii.gz"
  dseg_image = ants.image_read(op.join(data_dir, dseg_fname))

  # resample anatomical images to diffusion space
  dseg_image = ants.resample_image_to_target(dseg_image, mask_image, interp_type = "nearestNeighbor")

  # create WM mask from discrete segmentations
  dseg_values = dseg_image.numpy() == 0 # intialize with background
  for i in [3, 4, 24, 42, 43]: dseg_values = np.logical_or(dseg_values, dseg_image.numpy() == i)
  dseg_image = np.logical_not(dseg_values) * 1.0 # white matter values
  dseg_zoom  = dseg_image[x_min:x_max, y_min:y_max, z_slice]

  # load correlation^2 images (already in diffusionbut space)
  corr2_fname = f"{participant}_multi-shell_method-{method}_corr2.nii.gz"
  corr2_image = nib.load(op.join(data_dir, corr2_fname)).get_fdata()
  corr2_zoom  = corr2_image[x_min:x_max, y_min:y_max, z_slice].reshape((nx, ny, nz))
  corr2_zoom[dseg_zoom == 0] = 0 # remove not wm voxels

  # create correlation^2 image actor
  corr2_actor = actor.slicer(
    data            = corr2_zoom, 
    value_range     = corr2_range,
    lookup_colormap = corr2_lut
  )

  for split in ["split-1", "split-2"]: # for each split
    # read current participant fodf files
    fname = f"{participant}_multi-shell_method-{method}_{split}_fODF.nii.gz"
    image = nib.load(op.join(data_dir, fname)).get_fdata()
    image = image[x_min:x_max, y_min:y_max, z_slice, ...] 
    image[dseg_zoom == 0] = 0 # remove not wm voxels
    image = image.reshape((nx, ny, nz, image.shape[-1]))

    # create fodf actor
    fodf_actor = actor.odf_slicer(
      odfs     = image,
      sphere   = get_sphere("symmetric362"), 
      scale    = 0.8, 
      colormap = None # rgb
    )

    # define participant save directory
    save_dir = op.join(paths_save, participant)
    os.makedirs(save_dir, exist_ok = True)

    # save fodf glyph
    scene.add(corr2_actor) # add corr2 heatmap in the background
    scene.add(fodf_actor)  # add fodf actor to the scene
    scene.background((1, 1, 1)) # white background
    save_name = re.sub(".nii.gz", ".png", fname)
    window.record(
      scene    = scene, 
      out_path = op.join(save_dir, save_name), 
      size     = (2400, 2400)
    )
    print(f"Saved: {save_name}")
    scene.clear()

In [ ]:
n = corr2_lut.GetNumberOfAvailableColors()

fig, ax = plt.subplots(1, 1, figsize = (8, 8), tight_layout = True)
h = ax.imshow(np.zeros((n, 1)), vmin = 0, vmax = 100, cmap = corr2_cmap)
ax.set_xticks([]); ax.set_yticks([])
cbar = fig.colorbar(h, ax = ax, label = "Split-half fODF $r^{2}$")
cbar_ticks = np.linspace(0, 100, 7)
cbar_ticks_str = [f"{x:.0f}%" for x in cbar_ticks]
cbar.set_ticks(cbar_ticks); cbar.set_ticklabels(cbar_ticks_str)
plt.show()

save_name = "colorbar.svg"
fig.savefig(op.join(paths_save, save_name))
print(f"Saved: {save_name}")